# M1 Semantic Search Engine with Faiss and DistilBERT

## Objective

- Build a search engine using FAISS similarity search library and a pre-trained DistilBERT model from Transformers.


- On your search for an optimal document retrieval method in the CDC’s huge knowledge base you decide to implement a semantic search engine to overcome known limitations of statistical (TfIdf) full-text search. Its weaknesses stem from the fact that it relies on counting and matching words in a search query with documents in the database in the document. Even though modern full-text search engines do include, synonyms, for example, still there are many ways to express the same idea. You know that Transformers models excel at contextual learning, so you decide to apply transfer learning with pre-trained BERT models to see if you can make your search engine smarter.

## Create a new Jupyter Notebook and load all relevant Python libraries.

In [11]:
import json
from pprint import pprint
import faiss
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

## Open the provided JSON file called sentences.json. It contains a list of strings (sentences.)

In [2]:
# Load the documents
with open('data/sentences.json', 'r') as file:
    documents = json.load(file)
file.close()

## Use AutoTokenizer and AutoModel classes from Transformers library to load a pre-trained model from Transformers, along with the appropriate tokenizer.

In [3]:
# Load the a BERT model and a tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Create an empty inverted index with FAISS.

In [22]:
# Create a flat Faiss index
index = faiss.IndexIDMap(faiss.IndexFlatIP(768)) # the size of our vector space

## Write an encoder function that inputs a string and outputs a dense PyTorch tensor.

In [52]:
# Build a function that uses a BERT model to vectorize the texts
def encode(document):
    # Encode the documents and return vectors
    tokens = tokenizer(document, return_tensors='pt')
    vector = model(**tokens)[0].detach().squeeze()
    return torch.mean(vector, dim=0)

## Build a list of modeled vector representations for each document with a reusable encoder function you created in step 5.

In [27]:
# vectorize the documents
vectors = [encode(d) for d in documents]

In [28]:
[v.size() for v in vectors]

[torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768]),
 torch.Size([768])]

## Populate the empty FAISS index with the output vectors.

In [31]:
# Add the document vectors into the index. They need to be transformed into numpy arrays first
index.add_with_ids(
    np.array([v.numpy() for v in vectors]),
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(documents)))
)

## Build a search function that accepts a string query, encodes it, searches similar documents in the index, and returns top 5 results with their top_k scores.

In [29]:
# Build a function to search the index and return scored results
def search(query, k=5):
    # Search the index and return top scored results
    encoded_query = encode(query).unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [documents[_id] for _id in top_k[1][0]]
    results = list(zip(results, scores))
    return results

## Test your search engine by asking some questions. Check out the attached questions.json for a few suggested questions to start with, but feel free to play around and search for anything you want!

In [30]:
pprint(search("spanish flu casualties", k=2))

[('The Spanish flu, also known as the 1918 flu pandemic, was an unusually '
  'deadly influenza pandemic caused by the H1N1 influenza A virus.',
  51.06952),
 ('As of 2018, approximately 37.9 million people are infected with HIV '
  'globally.',
  45.203133)]


In [32]:
questions = ["How many people have died during Black Death?", "Which diseases can be transmitted by animals?", "Connection between climate change and a likelihood of a pandemic", "What is an example of a latent virus", "Viruses in nanotechnology", "Giant viruses classification", "What are the notable pandemic prevention organizations?", "How many leprosy outbreaks are known to happen?", "What are the geographic areas with the highest transmission of malaria?", "How to prevent the spread of viral infections?"]

In [35]:
for question in questions:
    pprint(question)
    pprint(search(question, k=1))

'How many people have died during Black Death?'
[('As of 2018, approximately 37.9 million people are infected with HIV '
  'globally.',
  52.61343)]
'Which diseases can be transmitted by animals?'
[('A pandemic is an epidemic of an infectious disease that has spread across a '
  'large region, for instance multiple continents or worldwide, affecting a '
  'substantial number of people.',
  54.049507)]
'Connection between climate change and a likelihood of a pandemic'
[('A pandemic is an epidemic of an infectious disease that has spread across a '
  'large region, for instance multiple continents or worldwide, affecting a '
  'substantial number of people.',
  60.54062)]
'What is an example of a latent virus'
[('A pandemic is an epidemic of an infectious disease that has spread across a '
  'large region, for instance multiple continents or worldwide, affecting a '
  'substantial number of people.',
  59.449497)]
'Viruses in nanotechnology'
[('Current pandemics include COVID-19 (SARS-Co

# M2 Searching Long Documents with Sentence Transformers

## Objective

- Implement a search engine using sentence-transformers and FAISS.


- In this milestone, instead of a base BERT model like we did previously, we will use Sentence BERT (SBERT). SBERT was developed to tackle similarity search and unsupervised clustering problems, for which classic BERT is not a good candidate.


- SBERT outputs fix embeddings for entire sentences and paragraphs instead of tokens and allow to drastically reduce computation time while keeping exceptional accuracy.


- You have successfully implemented your first prototype of a semantic similarity search engine using FAISS, a library for similarity search, and DistilBERT in Milestone 1 of this project. However, you are asking yourself what to do about the fact that most documents in the CDC’s database are much longer than the limit that this model can work with. Luckily, you’ve found out that there is another library called sentence-transformers that just might work for you! The models in this library have been developed and trained by the leading NLP researchers to extract meaningful embeddings for longer texts. Let’s test it out!

## Import the following libraries into a Jupyter or Colab Notebook:

- JSON
- FAISS
- sentence-transformers
- PyTorch
- NumPy

In [7]:
!cuda

True


In [8]:
from sentence_transformers import SentenceTransformer, util

In [9]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

## Load and open the provided data.json file.

In [16]:
# Load the documents
with open('data/data.json', 'r') as file:
    documents = json.load(file)
file.close()

In [17]:
documents[0]['text']

'A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.'

## Compute the sentence embeddings for documents in the data.json file.

In [18]:
corpus = [d['text'] for d in documents]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [19]:
np.array(corpus_embeddings).shape[1]

768

## Create an empty FAISS index and with the new documents exactly the same way as you did in M1.

In [42]:
embeddings = np.array(corpus_embeddings)

In [43]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

In [63]:
# vectors = [encode(d) for d in corpus]
# RuntimeError: The size of tensor a (668) must match the size of tensor b (512) at non-singleton dimension 1

In [44]:
index.add_with_ids(
    embeddings,
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(corpus)))
)

## Just like in Milestone 1, build a search function to retrieve the top 5 most similar documents.

In [45]:
D, I = index.search(np.array([embeddings[20]]), k=10)

In [46]:
D, I

(array([[184.57399 , 100.6426  ,  99.09663 ,  83.06282 ,  77.5573  ,
          75.87485 ,  75.54655 ,  75.020515,  74.69157 ,  74.33935 ]],
       dtype=float32),
 array([[20, 22, 10, 17, 21, 19, 25, 15,  0,  6]]))

In [47]:
# Build a search function that finds the most relevant search results
def search(query, corpus=corpus, k=5):
    # Compute query embeddings
    vector = embedder.encode(list(query))
    # Search top results in the Faiss index
    D, I = index.search(np.array(vector).astype("float32"), k=k)
    scores = D[0]
    results = [corpus[i] for i in I[0]]
    return list(zip(results, scores))

## Test the search function with a few queries. Use the provided questions.json for a few ideas.

In [35]:
query = "how many people died from black death?"

In [28]:
# Print out the results
query = "How many people died during Black plague?"
pprint(search(query, corpus, k=2))
# results=search(query)

# print('Top search results:')
# for result in results:
#     print(result)

[('Swine influenza is an infection caused by any one of several types of swine '
  'influenza viruses. Swine influenza virus (SIV) or swine-origin influenza '
  'virus (S-OIV) is any strain of the influenza family of viruses that is '
  'endemic in pigs. As of 2009, the known SIV strains include influenza C and '
  'the subtypes of  influenza A known as H1N1, H1N2, H2N1, H3N1, H3N2, and '
  'H2N3.\n'
  'Swine influenza virus is common throughout pig populations worldwide. '
  'Transmission of the virus from pigs to humans is not common and does not '
  'always lead to human flu, often resulting only in the production of '
  'antibodies in the blood. If transmission does cause human flu, it is called '
  'zoonotic swine flu. People with regular exposure to pigs are at increased '
  'risk of swine flu infection.\n'
  'Around the mid-20th century, identification of influenza subtypes became '
  'possible, allowing accurate diagnosis of transmission to humans. Since '
  'then, only 50 such

In [29]:
pprint(search("spanish flu casualties",corpus, k=2))

[('The Pandemic Severity Assessment Framework (PSAF) is an evaluation '
  'framework which uses quadrants to evaluate both the transmissibility and '
  'clinical severity of a pandemic and to combine these into an overall impact '
  'estimate.\n'
  'Clinical severity is calculated via multiple measures including case '
  'fatality rate, case-hospitalization ratios, and deaths-hospitalizations '
  'ratios, while viral transmissibility is measured via available data among '
  'secondary household attack rates, school attack rates, workplace attack '
  'rates, community attack rates, rates of emergency department and outpatient '
  'visits for influenza-like illness.The PSAF superseded the 2007 linear '
  'Pandemic Severity Index (PSI), which assumed 30% spread and measured case '
  'fatality rate (CFR) to assess the severity and evolution of the pandemic. '
  'The United States Centers for Disease Control and Prevention (CDC) adopted '
  'the PSAF as its official pandemic severity assess

In [36]:
answers = search("spanish flu casualties",corpus, k=2)

In [39]:
answers[0][0]

"The Pandemic Severity Assessment Framework (PSAF) is an evaluation framework which uses quadrants to evaluate both the transmissibility and clinical severity of a pandemic and to combine these into an overall impact estimate.\nClinical severity is calculated via multiple measures including case fatality rate, case-hospitalization ratios, and deaths-hospitalizations ratios, while viral transmissibility is measured via available data among secondary household attack rates, school attack rates, workplace attack rates, community attack rates, rates of emergency department and outpatient visits for influenza-like illness.The PSAF superseded the 2007 linear Pandemic Severity Index (PSI), which assumed 30% spread and measured case fatality rate (CFR) to assess the severity and evolution of the pandemic. The United States Centers for Disease Control and Prevention (CDC) adopted the PSAF as its official pandemic severity assessment tool in 2014, and it was the official pandemic severity assess

## Write the FAISS index to file. It will be useful for the next, and final, milestone!

In [30]:
faiss.write_index(index, "data/pandemics")

# M3 Question Answering with BERT

## Objective

- Create a question answering agent powered by sentence-transformers, FAISS similarity search engine, and a BERT model for question answering.


- In this milestone, we will try another BERT model (BERT for question answering) to see how we can not only retrieve a relevant text to the query but also derive the exact answer to our question from the passage. BERT models for question answering compute the probability of each token in a text to be the first or the last token in the answer to the question and return all tokens that are situated between the two, inclusively. This is called extractive question answering.


- As before, we need to be careful when choosing the model: It has to be finetuned for the specific task we are trying to perform, and we need to be mindful of the computational resources available to us when selecting it. The latter consideration is important to make sure that the model can complete this task in a reasonable time.


- You are pleased with your semantic search approach. Sentence-transformers return useful results and your CDC document search project is going full steam. This is really helping your colleagues to speed up their research and develop effective pandemic prevention procedures. Now your colleges have faith that you can help them be even more efficient by returning the relevant information in a more concise form. They hope you can make a question answering agent so that they don’t have to read the entire article to get an answer. You realized that Transformers library provides models trained and fine-tuned for question answering too. You decide to try it, and voila! Just like that, you’ve just built your very own mini-Google!

## Create a new notebook and import the required dependencies.

In [140]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer, util

## Load the FAISS document index you created and saved in Milestone 2.

In [101]:
# Load Faiss index cretaed in Milestone 2
index = faiss.read_index('./data/pandemics')

## Setup a transformers pipeline for question answering. 

We are going to use one of Transformers models, fine-tuned on SQUAD-v2 dataset (stands for Stanford Question Answering Dataset 2.0.)

In [141]:
# Load a transformers model
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
tokenizer = AutoTokenizer.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")
model = AutoModelForQuestionAnswering.from_pretrained("ktrapeznikov/albert-xlarge-v2-squad-v2")
# Load a question answering pipeline from Hugging Face Transformers
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [136]:
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""
result = nlp(question="What is extractive question answering?", context=context)
result

{'score': 0.5265803337097168,
 'start': 33,
 'end': 96,
 'answer': ' the task of extracting an answer from a text given a question.'}

## Rewrite the search function from Milestone 2 with an added functionality. 

It should encode a query, search within your FAISS library and extract answers from text using the question-answering pipeline.

In [129]:
# Modify the search function from Milestone 2
# to add question-answering functionality
def find_answer(query: str, k=5):
    # encode the query~
    encoded_embeddings = embedder.encode([query])
    # Search results in the Faiss index
    _, results = index.search(encoded_embeddings, k)
    # Use the transformers question-answering pipeline to find answer in text
    contexts = [corpus[i] for i in results[0]]
    answers = [nlp(question=query, context=context) for context in contexts]
    return sorted(answers, key=lambda x: x['score'], reverse=True)

In [135]:
#Checking for the query string which was used in Milestone1 and Milestone2 for comparison
pprint(find_answer("spanish flu casualties", k=2))

[{'answer': ' higher-than-expected mortality rate for young adults.',
  'end': 1373,
  'score': 0.07739856094121933,
  'start': 1319},
 {'answer': ' Actual fatalities ranged between 12,000 and 18,000.',
  'end': 1549,
  'score': 0.052253905683755875,
  'start': 1497}]


## Choose a query and search the FAISS index. 

Pass the search results along with your query to the question answering pipeline, and see how well it can extract and rank the answers.

In [137]:
query = "How to prevent the spread of viral infections?"
results=find_answer(query)

print('Top search results:')
for result in results:
    print(result)

Top search results:
{'score': 0.2111435830593109, 'start': 2751, 'end': 2762, 'answer': ' condom use'}
{'score': 0.06335747987031937, 'start': 899, 'end': 946, 'answer': '\nTracking viral load is used to monitor therapy'}
{'score': 0.05168566107749939, 'start': 217, 'end': 244, 'answer': ' administration of vaccines'}
{'score': 0.041904352605342865, 'start': 110, 'end': 119, 'answer': ' measures'}
{'score': 0.0002497527457308024, 'start': 1257, 'end': 1274, 'answer': ' Cholera vaccines'}


## Test your search engine by asking some questions. 

Check out the attached questions.json for a few suggested questions to start with, but feel free to play around and search for anything you want!

In [59]:
import random

In [142]:
# Load the questions
with open('data/questions.json', 'r') as file:
    questions = json.load(file)
file.close()

In [143]:
pprint([(query, find_answer(query)) for query in random.sample(questions, 2)])

[('Which diseases can be transmitted by animals?',
  [{'answer': ' infectious diseases',
    'end': 163,
    'score': 0.02271362394094467,
    'start': 143},
   {'answer': ' zoonotic swine flu',
    'end': 1051,
    'score': 0.0001700880820862949,
    'start': 1032},
   {'answer': ' pandemic',
    'end': 199,
    'score': 1.0785701306303963e-05,
    'start': 190},
   {'answer': ' unprotected sex (including anal and oral sex),',
    'end': 877,
    'score': 5.843516532877402e-07,
    'start': 830},
   {'answer': ' biological epidemic outbreaks,',
    'end': 286,
    'score': 3.396625345430948e-07,
    'start': 255}]),
 ('How many leprosy outbreaks are known to happen?',
  [{'answer': ' Seven',
    'end': 2554,
    'score': 0.12997889518737793,
    'start': 2548},
   {'answer': ' 31,156',
    'end': 1851,
    'score': 1.3111383850628044e-05,
    'start': 1844},
   {'answer': ' about 37.9 million',
    'end': 1689,
    'score': 6.183402547321748e-06,
    'start': 1670},
   {'answer': ' 26